In [ ]:
import xarray as xr
import numpy as np

In [ ]:
# Open pbp file
ds = xr.open_dataset('PbP_SODA/20250524_022503_F2DS_V.pbp.nc',decode_times=False)

### Saves probe time to UTC to display on images

In [ ]:
#convert seconds since midnight to hh:mm:ss
origin = np.datetime64('2025-05-24T00:00:00')
utc_times = origin + ds['probetime'].astype('timedelta64[s]')
ds['UTC_Time'] = utc_times

## Plot Raw image, no scaling or trimming

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Select a Particle Index (e.g., the 5000th particle) ---
particle_index = 34000

# --- 2. Extract Pointers ---
start_slice = ds['starty'].values[particle_index]
stop_slice = ds['stopy'].values[particle_index]
start_diode = ds['startx'].values[particle_index]
stop_diode = ds['stopx'].values[particle_index]

# --- 3. Extract the Image Sub-Array ---
# Slicing the large 'image' array using the extracted indices
particle_image = ds['image'].values[
    start_slice : stop_slice, 
    start_diode : stop_diode
]

# Get descriptive size metrics for the title
ysize_microns = ds['ysize'].values[particle_index]
xsize_microns = ds['xsize'].values[particle_index]

# --- 4. Plot the Image ---
plt.figure(figsize=(4, 6))
# 'binary' cmap shows 1s (shaded pixels) as black and 0s as white
plt.imshow(particle_image, cmap='binary', interpolation='none') 
plt.title(f"Particle {particle_index} | Size: {particle_image.shape[0]} slices x {particle_image.shape[1]} diodes")
plt.xlabel('Diode (X-Position)')
plt.ylabel('Slice (Y-Position / Along Airflow)')
plt.show()

## Plot histogram of phases for flight of interest

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# Open your NetCDF file
rf_ds = xr.open_dataset('RF05.20240311.051845_125704.PNI.nc')

# Replace 'temperature' with the actual variable name in your dataset
temp = rf_ds['ATX'].values

# Define bins: (-inf, -30], (-30, 0], (0, inf)
bins = [-np.inf, -40, 0, np.inf]
labels = ['Ice', 'Mixed Phase', 'Water']

# Bin the data
categories = pd.cut(temp, bins=bins, labels=labels)

# Count occurrences in each bin
hist = pd.value_counts(categories, sort=False)


hist.plot(kind='bar')
plt.xlabel('Temperature Range')
plt.ylabel('Count')
plt.title('Temperature Histogram')
plt.show()

## Standardized function to scale a particle to a 128x128 canvas and save the plot into specified output folder

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import os

# Make sure the output directory exists
output_dir = "cgwaves2_outputs"
os.makedirs(output_dir, exist_ok=True)

def plot_particle_standardized(ds, particle_index, target_size=128, max_fit=124):
    """
    Plot a single particle on a standardized square canvas.
    
    Parameters:
    -----------
    ds : xarray.Dataset
        Dataset containing particle image data
    particle_index : int
        Index of the particle to plot
    target_size : int
        Size of the output canvas (default: 128x128)
    max_fit : int
        Maximum dimension for the particle before padding (default: 124)
    
    Returns:
    --------
    canvas : np.ndarray
        The standardized 128x128 image array
    """
    
    # Extract bounding box
    start_slice = ds['starty'].values[particle_index]
    stop_slice = ds['stopy'].values[particle_index]
    start_diode = ds['startx'].values[particle_index]
    stop_diode = ds['stopx'].values[particle_index]
    probe_time=ds.UTC_Time[particle_index].values.astype('M8[ms]').astype('str')
    # Crop the particle image
    cropped_image = ds['image'].values[
        start_slice:stop_slice, 
        start_diode:stop_diode
    ]
    
    # Trim to only rows/cols with data
    rows_with_data = np.any(cropped_image == 1, axis=1)
    cols_with_data = np.any(cropped_image == 1, axis=0)
    
    if not np.any(rows_with_data) or not np.any(cols_with_data):
        print(f"Warning: Particle {particle_index} has no shaded pixels")
        return None
    
    cropped_image = cropped_image[rows_with_data][:, cols_with_data]
    H_current, W_current = cropped_image.shape
    
    # Calculate scaling factor (preserve aspect ratio)
    max_dim = max(H_current, W_current)
    scale_factor = max_fit / max_dim
    
    # Calculate new dimensions
    new_H = max(1, int(round(H_current * scale_factor)))
    new_W = max(1, int(round(W_current * scale_factor)))
    
    # Resize using nearest-neighbor to keep binary
    resized_image = resize(
        cropped_image,
        (new_H, new_W),
        order=0,
        anti_aliasing=False,
        preserve_range=True
    )
    resized_image = (resized_image > 0.5).astype(np.uint8)
    
    # Create canvas and center the particle
    canvas = np.zeros((target_size, target_size), dtype=np.uint8)
    pad_y = (target_size - new_H) // 2
    pad_x = (target_size - new_W) // 2
    canvas[pad_y:pad_y + new_H, pad_x:pad_x + new_W] = resized_image
    
    # Plot
    plt.figure(figsize=(6, 6))
    plt.imshow(canvas, cmap='binary', interpolation='none')
    plt.title(f"{probe_time} | Original: {H_current}×{W_current} → Scaled: {target_size}×{target_size}")
    plt.xlabel('X-Dimension')
    plt.ylabel('Y-Dimension')
    plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/particle_{particle_index}.png")
    plt.close()
    return canvas

# Example usage:
# canvas = plot_particle_standardized(ds, particle_index=34000)

In [ ]:
plot_particle_standardized(ds, particle_index=34000)

### Save images for all particles, or specify your start particle

In [ ]:
start_particle= 721600 ## specify where in array you want to start plotting

# Loop through all indices (or a subset for testing)
for particle_index in range(start_particle,ds.dims['Time']):
    plot_particle_standardized(ds, particle_index)